# Экспертная система - животные

В этом упражнении мы реализуем экспертную систему "животные" из одной классической книги. Дерево И-ИЛИ системы приведено на рисунке ниже:

![AND-OR Tree](https://raw.githubusercontent.com/shwars/AISchool/master/images/ExpSysAnimal.jpg)

## Обратный вывод

Попробуем реализовать простую экспертную систему обратного вывода самостоятельно. Для представления правил и запросов определим несколько классов:

In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x))
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices))
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Сами правила базы знаний можно записать следующим образом:

In [3]:
rules = {
    'default': Ask(['y','n']),
    'цвет' : Ask(['рыжевато-коричневый','чёрное с белым','другое']),
    'узор' : Ask(['чёрные полосы','тёмные пятна']),
    'млекопитающее': If(OR(['волосы','даёт молоко'])),
    'хищник': If(OR([AND(['острые зубы','когти','вперед смотрящие глаза']),'ест мясо'])),
    'копытное': If(['млекопитающее',OR(['копыта','жуёт жвачку'])]),
    'птица': If(OR(['оперение',AND(['летает','откладывает яйца'])])),
    'животное:обезьяна' : If(['млекопитающее','хищник','цвет:рыжевато-коричневый','узор:тёмные пятна']),
    'животное:тигр' : If(['млекопитающее','хищник','цвет:рыжевато-коричневый','узор:чёрные полосы']),
    'животное:жираф' : If(['копытное','длинная шея','длинные ноги','узор:тёмные пятна']),
    'животное:зебра' : If(['копытное','узор:чёрные полосы']),
    'животное:страус' : If(['птица','длинная шея','цвет:черное с белым','не может летать']),
    'животное:пингвин' : If(['птица','плавает','цвет:черное с белым','не может летать']),
    'животное:альбатрос' : If(['птица','хорошо летает'])
}

Наконец опишем рекурсивую машину обратного вывода:

In [20]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

In [21]:
kb = KnowledgeBase(rules)
kb.get('животное')

волосы
y/n
y
острые зубы
y/n
y
когти
y/n
y
вперед смотрящие глаза
y/n
y
цвет:рыжевато-коричневый
y/n
n
копыта
y/n
y
длинная шея
y/n
y
длинные ноги
y/n
y
узор:тёмные пятна
y/n
n
узор:чёрные полосы
y/n
y


'зебра'

## Прямой вывод

Прямой вывод мы реализуем на базе библиотеки [PyKnow](https://pyknow.readthedocs.io). **PyKnow** - это библиотека для построения экспертных систем прямого вывода на Python, которая спроектирована так, чтобы быть похожей на классическую систему [CLIPS](http://www.clipsrules.net/index.html). 

Наивную систему прямого вывода тоже не составило бы труда реализовать самим, однако она была бы неэффективной. Для реализации эффективного матчинга правил и состояния рабочей памяти используется алгоритм *Rete*.

In [2]:
import sys
!{sys.executable} -m pip install pyknow

  Running setup.py bdist_wheel for pyknow ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/96/ef/da/808b35ad3b3495cab3ca610cc8ce757280f1c36d529d0771de
  Running setup.py bdist_wheel for frozendict ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built pyknow frozendict


In [3]:
from pyknow import *

In [43]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('острые зубы'),Fact('когти'),Fact('вперёд смотрящие глаза')),
           Fact('ест мясо')))
    def cornivor(self):
        self.declare(Fact('хищник'))
        
    @Rule(OR(Fact('волосы'),Fact('даёт молоко')))
    def mammal(self):
        self.declare(Fact('млекопитающее'))

    @Rule(Fact('млекопитающее'),
          OR(Fact('копыта'),Fact('жуёт жвачку')))
    def hooves(self):
        self.declare('копытное')
        
    @Rule(OR(Fact('оперение'),AND(Fact('летает'),Fact('откладывает яйца'))))
    def bird(self):
        self.declare('птица')
        
    @Rule(Fact('млекопитающее'),Fact('хищник'),
          Fact(color='рыжевато-коричневый'),
          Fact(pattern='тёмные пятна'))
    def monkey(self):
        self.declare(Fact(animal='обезьяна'))

    @Rule(Fact('млекопитающее'),Fact('хищник'),
          Fact(color='рыжевато-коричневый'),
          Fact(pattern='чёрные полосы'))
    def tiger(self):
        self.declare(Fact(animal='тигр'))

    @Rule(Fact('копытное'),
          Fact('длинная шея'),
          Fact('длинные ноги'),
          Fact(pattern='тёмные пятна'))
    def giraffe(self):
        self.declare(Fact(animal='жираф'))

    @Rule(Fact('копытное'),
          Fact(pattern='чёрные полосы'))
    def zebra(self):
        self.declare(Fact(animal='зебра'))

    @Rule(Fact('птица'),
          Fact('длинная шея'),
          Fact('не может летать'),
          Fact(color='чёрное с белым'))
    def straus(self):
        self.declare(Fact(animal='страус'))

    @Rule(Fact('птица'),
          Fact('плавает'),
          Fact('не может летать'),
          Fact(color='чёрное с белым'))
    def pinguin(self):
        self.declare(Fact(animal='пингвин'))

    @Rule(Fact('птица'),
          Fact('хорошо летает'))
    def albatros(self):
        self.declare(Fact(animal='альбатрос'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Животное - {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

In [47]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='рыжевато-коричневый'),
    Fact(pattern='чёрные полосы'),
    Fact('острые зубы'),
    Fact('когти'),
    Fact('вперёд смотрящие глаза'),
    Fact('даёт молоко')])
ex1.run()
ex1.facts

Животное - тигр


FactList([(0, InitialFact()),
          (1, Fact(color='рыжевато-коричневый')),
          (2, Fact(pattern='чёрные полосы')),
          (3, Fact('острые зубы')),
          (4, Fact('когти')),
          (5, Fact('вперёд смотрящие глаза')),
          (6, Fact('даёт молоко')),
          (7, Fact('млекопитающее')),
          (8, Fact('хищник')),
          (9, Fact(animal='тигр'))])